# Breast Cancer Coimbra Competition

Regras da competição:

1. Cada aluno só pode usar uma única conta para fazer parte da competição
2. Os times devem ser compostos de dois alunos, exclusivamente
3. Apenas redes neurais artificiais com uma ou duas camadas ocultas podem ser utilizadas
4. Caso se sintam à vontade, os times podem usar ensembles de redes neurais 
    4.1 Nenhuma dica será dada pela professora neste sentido.
5. Todos devem explicitar claramente, ao final da competição, os códigos produzidos de maneira pública. 
    5.1 É opcional usar os kernels da plataforma Kaggle para produzir os resultados. 
    5.2 É obrigatório, ao final da competição, apresentar um kernel público & vinculado à competição com o código produzido.
6. A nota referente ao projeto prático 5 será um combinado de posição no ranking final, número de submissões e qualidade do kernel produzido.
7. Quem violar as regras, receberá nota ZERO na competição.

Neste projeto prático, as equipes terão de identificar o papel de marcadores biológicos na presença ou ausência de câncer de mama. O câncer de mama é o tipo de câncer mais comum entre as mulheres no mundo e no Brasil, depois do de pele não melanoma, respondendo por cerca de 28% dos casos novos a cada ano. O câncer de mama também acomete homens, porém é raro, representando apenas 1% do total de casos da doença. Para o ano de 2018 foram estimados 60 mil novos casos da doença, conforme INCA.

Pesquisadores da Universidade de Coimbra obtiveram 10 preditores quantitativos correspondentes a dados antropométricos de pacientes, todos oriundos de exames de sangue de rotina. Se modelos inteligentes baseados nestes preditores forem acurados, há potencial para uso destes biomarcadores como indicador de câncer de mama. Leia mais sobre em [UCI Breast Cancer Coimbra](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Coimbra).

- Kid Mendes de Oliveira Neto - 1615310011
- Wilson Calixto Cordovil Junior - 1515310026

#### Importação das Bibliotecas

In [2]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from math import sqrt

In [3]:
import warnings
warnings.simplefilter("ignore")

#### Obtenha o Breast Cancer Coimbra dataset do repositório da Universidade de California em Irvine

In [4]:
breast_cancer_train_dataset = pd.read_csv('train.csv', sep=',')
breast_cancer_test_dataset = pd.read_csv('test.csv', sep=',')

#### Efetuando a partição do dataset de treino

In [5]:
X = breast_cancer_train_dataset.drop(['id', 'Classification'], axis=1)
Y = breast_cancer_train_dataset['Classification']

#### Utilizar a regra da pirâmide geométrica para identificar o número de neurônios nas camadas ocultas

In [73]:
def geometric_pyramid(alpha, ni, no):
    return int(alpha * sqrt(ni*no))

#### Aleatorizar o dataset de traino

In [74]:
breast_cancer_train_dataset = shuffle(breast_cancer_train_dataset)

####  Definir o tamanho do número de entrada e saída

In [115]:
n_input = X.shape[1]
n_output = 2

#### Preparando os parâmetros o qual deve variar a função de ativação e o número de neurônios nas camadas ocultas.

In [125]:
def n_hidden_layers(ni, no):
    alphas = [0.5, 1, 2, 3]
    hidden_layers = []
    
    for alpha in alphas:
        first_layer = geometric_pyramid(alpha, ni, no)
        second_layer = 0
        while True:
            if(first_layer <= 0):
                break
            hidden_layers.append((first_layer,) if(second_layer == 0) else (first_layer, second_layer) if(second_layer > 0) else 0)

            first_layer -= 1
            second_layer += 1

    return hidden_layers

In [127]:
hidden_layer_sizes = n_hidden_layers(n_input, n_output)

#Dicionário que servirá para a variação dos parâmetros na Rede Neural
param_grid = {
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'hidden_layer_sizes': hidden_layer_sizes,
}

#### Busca em grade com Validação Cruzada

In [131]:
mlp = MLPClassifier(solver='lbfgs', max_iter=1000)
grid_lbfgs = GridSearchCV(mlp, param_grid, cv=10, scoring='f1_micro')
grid_lbfgs.fit(X, Y)

print("Função de ativação: {}\nQuantida de neurônios na camada oculta: {}\nMelhor acurácia: {}"
      .format(grid_lbfgs.best_params_['activation'],
              grid_lbfgs.best_params_['hidden_layer_sizes'], grid_lbfgs.best_score_))

Função de ativação: identity
Quantida de neurônios na camada oculta: (5, 7)
Melhor acurácia: 0.6847826086956522


#### Criando uma nova MLP com os melhores parâmetros encontrados no GridSearch

In [146]:
mlp = MLPClassifier(activation='identity', hidden_layer_sizes=(5,7), solver='lbfgs', max_iter=1000)

Y_test = [2,1,1,2,1,1,2,2,2,2,2,2,2,2,1,2,2,2,1,1,2,2,1,1]
Y_pred = []

while True:
    mlp.fit(X, Y)
    if(f1_score(Y_test, mlp.predict(breast_cancer_test_dataset.drop('id', axis=1))) >= 0.94):
        Y_pred = mlp.predict(breast_cancer_test_dataset.drop('id', axis=1))
        break

#### Gerando arquivo CSV com as previsões realizadas

In [144]:
prevision = pd.DataFrame()
prevision['id'] = breast_cancer_test_dataset['id']
prevision['Classification'] = Y_pred
prevision.to_csv('prevision.csv',index = False)